In [ ]:
from transformers import AutoImageProcessor, AutoModelForSemanticSegmentation
import matplotlib.pyplot as plt
import os
import utils
from dataset import XRayDataset
import numpy as np
from torch.utils.data import DataLoader

In [12]:
def encode_mask_to_rle(mask):
    '''
    mask: numpy array binary mask 
    1 - mask 
    0 - background
    Returns encoded run length 
    '''
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# RLE로 인코딩된 결과를 mask map으로 복원합니다.

def decode_rle_to_mask(rle, height, width):
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)

In [ ]:
model_dir_path = '/data/ephemeral/home/Dongjin/level2-cv-semanticsegmentation-cv-02-lv3/Baseline/Dongjin/transformers_1120/trained_models/upernet-convnext-small_cont_size_1024'

conf_path = os.path.join(model_dir_path, 'exp.json')
conf = utils.read_json(conf_path)
conf['step_batch_size'] = 1

saved_model_dir_path = utils.saved_model_dir_path(model_dir_path)

image_processor = AutoImageProcessor.from_pretrained(saved_model_dir_path)
model = AutoModelForSemanticSegmentation.from_pretrained(saved_model_dir_path).to(conf['device'])

train_dataset = XRayDataset(mode='train', 
                            transforms=None, 
                            image_processor=image_processor,
                            data_dir_path=conf['data_dir_path'],
                            data_info_path=conf['train_json_path'])

valid_dataset = XRayDataset(mode='valid', 
                            transforms=None, 
                            image_processor=image_processor,
                            data_dir_path=conf['data_dir_path'],
                            data_info_path=conf['valid_json_path'])

test_dataset = XRayDataset(mode='test', 
                            transforms=None, 
                            image_processor=image_processor,
                            data_dir_path=conf['data_dir_path'],
                            data_info_path=conf['test_json_path'])

train_loader = DataLoader(train_dataset, batch_size=conf['step_batch_size'], shuffle=True, num_workers=conf['num_workers'])
valid_loader = DataLoader(valid_dataset, batch_size=conf['step_batch_size'], shuffle=False, num_workers=conf['num_workers'])
test_loader = DataLoader(test_dataset, batch_size=conf['step_batch_size'], shuffle=False, num_workers=conf['num_workers'])

{'pixel_values': tensor([[[[-1.6898, -1.6727, -1.6727,  ...,  2.2318,  2.2318,  2.2318],
          [-1.6898, -1.6555, -1.6727,  ...,  2.2318,  2.2318,  2.2318],
          [-1.7240, -1.7069, -1.7240,  ...,  2.2318,  2.2318,  2.2318],
          ...,
          [-1.5014, -1.4843, -1.5185,  ..., -1.9124, -1.8953, -1.9124],
          [-1.4500, -1.4500, -1.4843,  ..., -1.9295, -1.9295, -1.9295],
          [-1.4500, -1.4672, -1.4500,  ..., -1.9467, -1.9467, -1.9467]],

         [[-1.5980, -1.5805, -1.5805,  ...,  2.4111,  2.4111,  2.4111],
          [-1.5980, -1.5630, -1.5805,  ...,  2.4111,  2.4111,  2.4111],
          [-1.6331, -1.6155, -1.6331,  ...,  2.4111,  2.4111,  2.4111],
          ...,
          [-1.4055, -1.3880, -1.4230,  ..., -1.8256, -1.8081, -1.8256],
          [-1.3529, -1.3529, -1.3880,  ..., -1.8431, -1.8431, -1.8431],
          [-1.3529, -1.3704, -1.3529,  ..., -1.8606, -1.8606, -1.8606]],

         [[-1.3687, -1.3513, -1.3513,  ...,  2.6226,  2.6226,  2.6226],
          [-1